[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/iceberg12/vietai-anlp/blob/main/Week1_CBOW-TruyenKieu.ipynb)

In [1]:
!wget --no-check-certificate \
    https://gist.githubusercontent.com/khacanh/4c4662fa226db87a4664dfc2f70bc63e/raw/5d8a1d890c73a1e92e6898137db28f3dc0676975/kieu.txt \
    -O ./kieu.txt

--2021-01-10 01:28:54--  https://gist.githubusercontent.com/khacanh/4c4662fa226db87a4664dfc2f70bc63e/raw/5d8a1d890c73a1e92e6898137db28f3dc0676975/kieu.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 135832 (133K) [text/plain]
Saving to: ‘./kieu.txt’

./kieu.txt          100%[===================>] 132.65K  --.-KB/s    in 0.04s   

2021-01-10 01:28:54 (3.46 MB/s) - ‘./kieu.txt’ saved [135832/135832]



In [34]:
corpus = []
f = open("kieu.txt", "r")
for line in f:
  corpus.append(line)

In [35]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence

In [36]:
import tensorflow as tf


In [37]:
for i in range(len(corpus)):
  corpus[i] = text_to_word_sequence(corpus[i])


In [38]:
corpus[:20]

[['trăm', 'năm', 'trong', 'cõi', 'người', 'ta'],
 ['chữ', 'tài', 'chữ', 'mệnh', 'khéo', 'là', 'ghét', 'nhau'],
 ['trải', 'qua', 'một', 'cuộc', 'bể', 'dâu'],
 ['những', 'điều', 'trông', 'thấy', 'mà', 'đau', 'đớn', 'lòng'],
 ['lạ', 'gì', 'bỉ', 'sắc', 'tư', 'phong'],
 ['trời', 'xanh', 'quen', 'thói', 'má', 'hồng', 'đánh', 'ghen'],
 ['cảo', 'thơm', 'lần', 'giở', 'trước', 'đèn'],
 ['phong', 'tình', 'có', 'lục', 'còn', 'truyền', 'sử', 'xanh'],
 ['rằng', 'năm', 'gia', 'tĩnh', 'triều', 'minh'],
 ['bốn', 'phương', 'phẳng', 'lặng', 'hai', 'kinh', 'vững', 'vàng'],
 ['có', 'nhà', 'viên', 'ngoại', 'họ', 'vương'],
 ['gia', 'tư', 'nghĩ', 'cũng', 'thường', 'thường', 'bực', 'trung'],
 ['một', 'trai', 'con', 'thứ', 'rốt', 'lòng'],
 ['vương', 'quan', 'là', 'chữ', 'nối', 'dòng', 'nho', 'gia'],
 ['đầu', 'lòng', 'hai', 'ả', 'tố', 'nga'],
 ['thúy', 'kiều', 'là', 'chị', 'em', 'là', 'thúy', 'vân'],
 ['mai', 'cốt', 'cách', 'tuyết', 'tinh', 'thần'],
 ['một', 'người', 'một', 'vẻ', 'mười', 'phân', 'vẹn', 'mười'],


In [39]:
import random
idx = list(range(len(corpus)))
random.Random(12).shuffle(idx)
test_ratio = 0.1
test_idx = idx[:int(test_ratio*len(idx))]
train_idx = idx[int(test_ratio*len(idx)):]
corpus_train = [corpus[i] for i in train_idx]
corpus_test = [corpus[i] for i in test_idx]


In [40]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(corpus_train)
w2id = tokenizer.word_index

In [41]:
len(corpus_train)

2929

In [42]:
len(w2id)

2338

In [49]:
w2id['<OOV>']

1

In [43]:
from tensorflow.keras.utils import to_categorical

In [44]:
vocab_size = len(tokenizer.word_index) + 1
window_size = 2

In [50]:
import numpy as np
def generate_pairs(window_size, corpus):

  X = []
  y = []
  for words in corpus:
    start = 0
    while start + window_size * 2 < len(words):
      end = start + window_size * 2
      tar_i = start + window_size
      x = [" ".join(words[start:tar_i] + words[tar_i+1:end+1])]
      label = words[tar_i]
      # print(words)
      # print(x, "--->", label)
      start += 1
      X.append(tokenizer.texts_to_sequences(x)[0])
      y.append(to_categorical( w2id.get(label, w2id['<OOV>']), len(w2id) + 1))

  return tf.convert_to_tensor(X) , tf.convert_to_tensor(y)


In [51]:
X_train, y_train = generate_pairs(window_size, corpus_train)
X_test, y_test = generate_pairs(window_size, corpus_test)


In [52]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Lambda
from tensorflow.keras.backend import mean 


In [53]:
X_train[0], y_train[0]

(<tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 137,  121, 1653,   92], dtype=int32)>,
 <tf.Tensor: shape=(2339,), dtype=float32, numpy=array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)>)

In [54]:
vocab_size

2339

In [55]:


# vocab_size x embedding_size



In [56]:
embedding_size = 128
cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=window_size*2))
cbow.add(Lambda(lambda x: mean(x, axis=1), output_shape=(embedding_size,) ))
cbow.add(Dense(vocab_size, activation='softmax'))

cbow.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [57]:
cbow.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 4, 128)            299392    
_________________________________________________________________
lambda (Lambda)              (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 2339)              301731    
Total params: 601,123
Trainable params: 601,123
Non-trainable params: 0
_________________________________________________________________


In [58]:
cbow.fit(X_train, y_train, epochs=30, verbose=1, validation_data=(X_test,y_test))

Epoch 1/30
275/275 [==============================] - 4s 7ms/step - loss: 7.6829 - acc: 0.0153 - val_loss: 7.1157 - val_acc: 0.0195
Epoch 2/30
275/275 [==============================] - 2s 6ms/step - loss: 6.8239 - acc: 0.0207 - val_loss: 6.9755 - val_acc: 0.0195
Epoch 3/30
275/275 [==============================] - 2s 6ms/step - loss: 6.6276 - acc: 0.0203 - val_loss: 6.9761 - val_acc: 0.0195
Epoch 4/30
275/275 [==============================] - 2s 6ms/step - loss: 6.5248 - acc: 0.0271 - val_loss: 6.9742 - val_acc: 0.0215
Epoch 5/30
275/275 [==============================] - 2s 6ms/step - loss: 6.3753 - acc: 0.0334 - val_loss: 6.9617 - val_acc: 0.0287
Epoch 6/30
275/275 [==============================] - 2s 6ms/step - loss: 6.2689 - acc: 0.0446 - val_loss: 6.9459 - val_acc: 0.0318
Epoch 7/30
275/275 [==============================] - 2s 6ms/step - loss: 6.0433 - acc: 0.0625 - val_loss: 6.9148 - val_acc: 0.0369
Epoch 8/30
275/275 [==============================] - 2s 6ms/step - loss: 5.

In [61]:
example = 'Trăm năm cõi người'
example = text_to_word_sequence(example)
example = tokenizer.texts_to_sequences([example])
example = tf.convert_to_tensor(example)
cbow.predict(example)
tokenizer.index_word[np.argmax(cbow.predict(example))]

'trong'

In [63]:
corpus_test[0]

['thành', 'thân', 'mới', 'rước', 'xuống', 'thuyền']

In [64]:
example = 'thân mới xuống thuyền'
example = text_to_word_sequence(example)
example = tokenizer.texts_to_sequences([example])
example = tf.convert_to_tensor(example)
cbow.predict(example)
tokenizer.index_word[np.argmax(cbow.predict(example))]

'biết'

In [ ]:
weights = cbow.get_weights()[0]

In [ ]:
weights

array([[ 0.0318269 , -0.01505174,  0.03639983, ..., -0.02388797,
        -0.04805097,  0.01071918],
       [-0.05085587,  0.09169063,  0.03570038, ...,  0.0095367 ,
         0.02253897, -0.01499532],
       [ 0.03044103, -0.05055906,  0.0389102 , ...,  0.02233716,
         0.07572004,  0.02428444],
       ...,
       [ 0.02515461,  0.03420826, -0.04985545, ...,  0.0061751 ,
        -0.08323187, -0.09447721],
       [ 0.04186383, -0.04836989, -0.00351331, ..., -0.04674897,
        -0.07837866,  0.02577955],
       [ 0.06217309, -0.00417627,  0.06007205, ..., -0.06049352,
        -0.01882677,  0.02589382]], dtype=float32)

In [ ]:
vocab_size

118

In [ ]:
weights.shape

(117, 128)

In [ ]:
len(list(tokenizer.index_word.values()))

117

In [ ]:
import pandas as pd
weights = cbow.get_weights()[0]
weights = weights[1:]
print(weights.shape)

pd.DataFrame(weights, index=list(tokenizer.index_word.values())).head()

(117, 128)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127
<OOV>,0.031827,-0.015052,0.036400,-0.025224,-0.026220,-0.046286,0.019774,-0.008172,0.006869,0.012888,0.019361,0.016363,-0.022348,-0.049530,0.021561,-0.034322,0.035252,-0.036651,-0.034132,-0.047134,0.015056,-0.044091,-0.032397,-0.015606,-0.032107,0.016883,-0.028068,0.035492,-0.046504,0.016070,0.015872,-0.006604,-0.015332,0.014879,-0.032317,-0.009584,0.030967,-0.001324,0.001430,0.029867,...,0.036813,0.003025,0.030064,0.048660,-0.017831,0.045077,-0.000642,0.002372,-0.028924,-0.009973,0.027664,-0.042066,0.033804,-0.006118,-0.014136,-0.029626,0.004874,0.000314,-0.007406,0.028832,-0.011241,0.031732,-0.001889,-0.021923,0.023346,0.045387,0.017919,0.013830,0.029240,-0.042154,-0.014561,-0.044820,0.049010,-0.000719,-0.003991,-0.015966,0.018115,-0.023888,-0.048051,0.010719
là,-0.050856,0.091691,0.035700,0.079347,-0.067342,-0.055625,-0.062175,-0.068485,-0.007364,0.091895,-0.037921,0.014997,-0.043217,-0.080084,-0.028372,-0.034511,0.000644,-0.030253,-0.039692,-0.082604,0.048632,0.053437,0.072946,0.016293,0.101095,-0.018682,-0.012524,0.092336,0.018907,-0.002526,0.056696,0.054626,0.058131,-0.081643,-0.059354,-0.013012,-0.026246,-0.020010,-0.079771,-0.033148,...,0.079048,-0.083061,-0.075608,0.020159,0.073923,-0.069906,-0.094949,0.045342,-0.072858,0.009398,0.055825,0.109063,0.015088,-0.037631,-0.009864,0.049500,-0.037078,-0.013594,0.014778,0.015310,-0.010169,-0.076756,0.014949,0.053401,-0.109620,0.058109,0.104437,0.056287,-0.011091,0.027131,0.013445,-0.013708,-0.002052,-0.041543,-0.058824,0.051688,-0.015018,0.009537,0.022539,-0.014995
một,0.030441,-0.050559,0.038910,-0.055022,-0.098898,-0.072563,-0.056924,-0.085691,0.045195,-0.011041,-0.029022,-0.031123,-0.015146,0.002481,-0.077833,-0.058809,0.037818,0.037354,-0.018249,-0.008307,0.025979,0.037069,0.067813,0.093914,-0.071382,-0.047962,0.019437,-0.019180,0.010277,-0.029073,-0.041010,0.026244,-0.015512,-0.015577,-0.026633,-0.069078,-0.036043,-0.002754,-0.087668,-0.039390,...,0.071252,0.015932,0.040472,-0.019300,0.040164,-0.083157,0.025478,0.043400,0.019925,-0.010717,0.069204,0.022896,-0.054678,0.048628,-0.018888,0.057378,0.058815,-0.052711,-0.099871,-0.045400,0.024025,-0.067533,-0.049321,-0.093735,0.032452,-0.029155,0.028171,-0.084994,0.018872,0.004752,0.011821,0.085760,-0.064696,0.021297,-0.080001,-0.007060,-0.032212,0.022337,0.075720,0.024284
chữ,0.080730,0.007994,0.032952,0.025223,-0.025180,0.038352,0.014387,0.048172,-0.084220,0.020160,0.009457,0.048622,0.018438,-0.046196,-0.009081,-0.082187,-0.072830,-0.067188,0.078864,-0.032441,0.039488,0.017785,0.095161,-0.052993,0.092794,-0.079851,-0.001721,0.065388,0.014285,0.038979,0.063497,0.078690,-0.006945,0.006516,-0.055228,0.068180,-0.016177,-0.046261,-0.043580,-0.038526,...,0.072969,-0.038346,-0.008955,-0.063560,-0.056107,-0.045130,-0.033207,0.062682,-0.085406,0.088364,0.046721,0.044608,0.062543,0.000458,0.039971,-0.021890,-0.094858,0.032064,-0.010576,-0.014330,-0.100503,0.059387,0.045339,-0.073188,-0.086991,0.082181,0.045117,0.012198,-0.089433,0.061574,-0.045585,0.048226,0.031074,-0.023812,0.006986,0.026475,-0.064078,-0.030370,0.018127,0.000489
lòng,-0.030587,0.031163,-0.028113,-0.010189,-0.071232,-0.074625,-0.062832,0.069700,0.066684,0.023029,0.051540,0.083328,-0.024999,0.004759,0.021449,0.005331,0.037940,-0.015523,-0.040194,0.046623,0.033482,0.004455,0.060771,-0.006937,0.000809,-0.000585,-0.029694,-0.006711,0.014048,-0.091399,0.000662,0.009510,0.029037,0.108462,0.090062,-0.035629,-0.056562,-0.046336,-0.091317,-0.073941,...,0.016552,-0.027864,-0.075989,0.013949,0.044220,-0.063157,-0.081937,0.039825,0.011472,0.103083,0.048873,-0.050240,-0.001817,-0.074513,-0.001636,-0.041891,-0.050425,-0.061419,-0.033863,-0.055179,0.031185,0.063117,0.028971,-0.039968,0.018519,0.086755,-0.064150,0.002739,0.05

In [ ]:
weights.shape

(117, 128)